# Make Supplemental Tables and variations

This notebook combines dfs with p-values and correlations for 8 cancers. It makes the supplmental data table as well as the dataframes used in downstream anaylisis.  
Description of created csv files: 
* Supplemental_Table_2  - This is the supplemental table provided with the manuscript. It has the FDR corrected p-values and correlations for all proteins, 
* Supplemental_Table_EGFR_sig_only - This is the filtered version of suppl table 1 which only has FDR significant p-values
* all_heatmap - all data appended to make a long table for easy use with heatmap function, 
* sig_pval_heatmap - contains only significant genes in long format for heatmap, 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 


Read in the single cancer dfs with FDR corrected p-values and correlation values. Merge all dfs into one pancancer data frame.

In [4]:
Gbm_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_GBM')
Hnscc_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_HNSCC')
Luad_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_LUAD')
Lscc_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_LSCC')
Brca_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_BR')
Ovarian_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_OV')
Colon_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_CO')
Kidney_df = pd.read_csv('csv_files/trans_effects_all_prot_fdr_corrected_ccRCC')


# Make Supplemental Table 2

Merge all cancer data frames into one wide data frame

In [5]:
pancan = pd.merge(Gbm_df, Kidney_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Ovarian_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Luad_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Lscc_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Brca_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Colon_df, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Hnscc_df, on="Comparison", how = "outer")
pancan = pancan[1:]
pancan.to_csv('csv_files/Supplemental_Table_2.csv', index=False)
pancan.head()

,Comparison,Correlation_GBM,P_value_GBM,Correlation_ccRCC,P_value_ccRCC,Correlation_OV,P_value_OV,Correlation_LUAD,P_value_LUAD,Correlation_LSCC,P_value_LSCC,Correlation_BR,P_value_BR,Correlation_CO,P_value_CO,Correlation_HNSCC,P_value_HNSCC
1,PHLDA1,0.789551,9.649849e-19,0.180866,0.249285,0.383373,0.128340,0.257653,0.062170,0.682116,1.398013e-12,0.269548,0.016601,0.437505,0.143566,0.583064,1.395609e-08
2,CDH4,0.656331,4.744022e-10,0.015524,0.965357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GRB2,-0.589008,3.310325e-07,-0.245694,0.081461,-0.097221,0.639167,-0.269432,0.048945,-0.098621,5.999783e-01,-0.212609,0.066821,-0.017903,0.972007,-0.481605,2.053150e-05
4,SOCS2,0.565367,2.006861e-06,NaN,NaN,NaN,NaN,0.040982,0.873486,0.279846,7.403938e-02,-0.232949,0.302917,NaN,NaN,NaN,NaN
5,PHLDA3,0.561528,2.006861e-06,0.364164,0.005159,-0.260799,0.159831,0.215877,0.132154,0.651618,5.103796e-11,0.115916,0.361693,0.161425,0.624900,0.693125,2.351012e-14


# Make Supplemental_Table_EGFR_sig_only

In [6]:
Gbm_df_sig =Gbm_df.loc[(Gbm_df["P_value_GBM"] <= 0.05)]
Kidney_df_sig =Kidney_df.loc[(Kidney_df["P_value_ccRCC"] <= 0.05)]
Colon_df_sig =Colon_df.loc[(Colon_df["P_value_CO"] <= 0.05)]
Ovarian_df_sig =Ovarian_df.loc[(Ovarian_df["P_value_OV"] <= 0.05)]
Luad_df_sig =Luad_df.loc[(Luad_df["P_value_LUAD"] <= 0.05)]
Lscc_df_sig =Lscc_df.loc[(Lscc_df["P_value_LSCC"] <= 0.05)]
Brca_df_sig =Brca_df.loc[(Brca_df["P_value_BR"] <= 0.05)]
Hnscc_df_sig =Hnscc_df.loc[(Hnscc_df["P_value_HNSCC"] <= 0.05)]

In [7]:
pancan = pd.merge(Gbm_df_sig, Kidney_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Ovarian_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Luad_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Lscc_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Brca_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Colon_df_sig, on="Comparison", how = "outer")
pancan = pd.merge(pancan, Hnscc_df_sig, on="Comparison", how = "outer")
pancan = pancan[1:]
pancan.to_csv('csv_files/Supplemental_Table_EGFR_sig_only.csv', index=False)


# Make All Heatmap

In [8]:
# Create long df for heatmap

cancer = ['GBM','HNSCC','LSCC','LUAD','BR','ccRCC']
merged_dfs = [Gbm_df,Hnscc_df,Lscc_df,Luad_df,Brca_df,Kidney_df]

all_long_df = pd.DataFrame()
i = 0
for c in cancer:
    m = merged_dfs[i]
    m2 = m.assign(Cancer = c)
    m2 = m2.rename(columns={'P_value_' + c: 'P_Value'})
    m2 = m2.rename(columns={'Correlation_' + c: 'Correlation'})
    all_long_df = all_long_df.append(m2) 
    if i < 7:
        i += 1
        


In [9]:
all_long_df.to_csv('csv_files/all_prot_heatmap_EGFR.csv', index=False)
all_long_df

,Comparison,Correlation,P_Value,Cancer
0,EGFR,1.000000,0.000000e+00,GBM
1,PHLDA1,0.789551,9.649849e-19,GBM
2,CDH4,0.656331,4.744022e-10,GBM
3,GRB2,-0.589008,3.310325e-07,GBM
4,SOCS2,0.565367,2.006861e-06,GBM
...,...,...,...,...
10081,RPS29,0.000111,9.997098e-01,ccRCC
10082,YBX1,0.000072,9.997098e-01,ccRCC
10083,UBL7,0.000063,9.997098e-01,ccRCC
10084,HEATR3,0.000043,9.997470e-01,ccRCC


# Significant P values of Heatmap df 

In [10]:
only_sig_pvals = all_long_df.loc[(all_long_df["P_Value"] <= 0.05)]
only_sig_pvals.to_csv('csv_files/sig_prot_heatmap_EGFR.csv', index=False)